In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!nvidia-smi

Sun Jan 19 09:07:46 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install markitdown langchain chromadb gradio sentence-transformers
!pip install bitsandbytes accelerate git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.7 MB/s eta 0:0

In [ ]:
!pip install -U langchain-community
import os
from markitdown import MarkItDown
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import torch
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("Environment setup is complete and logging is configured.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.8 MB/s eta 0:00:00


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<fro

In [ ]:


import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load the Q&A data from Excel
file_path = "/content/lawbotica_questions.xlsx"
data = pd.read_excel(file_path)

# Clean and prepare the Q&A data
data.columns = ["Questions", "Answers", "Source"]  # Rename columns for consistency
questions = data["Questions"].dropna().tolist()
answers = data["Answers"].dropna().tolist()
sources = data["Source"].dropna().tolist()


print(f"Loaded {len(questions)} questions from the dataset!")

# Model name
model_name = "tiiuae/falcon-7b-instruct"


quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


tokenizer = AutoTokenizer.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)
model.eval()

print("✅ Falcon 7B Instruct loaded and ready!")


for i in range(min(3, len(questions))):
    print(f"Q{i+1}: {questions[i]}")
    print(f"A{i+1}: {answers[i]}")
    print(f"Source: {sources[i]}")
    print("-" * 40)


Loaded 76 questions from the dataset!


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

✅ Falcon 7B Instruct loaded and ready!
Q1: Questions
A1: Answers 
Source: Source
----------------------------------------
Q2: What is Digitally Derived Evidence (DDE) and how does it differ from regular digital evidence?
A2: According to the Leiden Guidelines, DDE encompasses both 'digital evidence' (material that is "born-digital" originating from a computer environment) and 'digitized evidence' (analogue material transferred to digital format). The concept includes evidence created by digital technology that records or traces events, as well as evidence that has been converted into digital form for storage and presentation purposes.
Source: PDF version https://leiden-guidelines.netlify.app/resources/downloads/
----------------------------------------
Q3: When submitting video evidence in international criminal courts, what are the key requirements regarding completeness and excerpts?
A3: According to Guideline A.1, videos should be submitted in their entirety rather than as excerpts 

In [ ]:
# Directory containing legal documents
DOCS_FOLDER = "/content/legal_documents"
DB_PATH = "/content/legal_chroma_db"


doc_files = [
    f for f in os.listdir(DOCS_FOLDER)
    if f.lower().endswith(('.pdf', '.docx', '.txt', '.html', '.pptx'))
]


def convert_and_split(file_path):
    md = MarkItDown()
    result = md.convert(file_path)
    text_content = result.text_content


    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = splitter.split_text(text_content)
    return chunks

texts = []
metadatas = []


for doc_name in doc_files:
    file_path = os.path.join(DOCS_FOLDER, doc_name)
    logger.info(f"Processing {doc_name}...")
    try:

        chunks = convert_and_split(file_path)
        for chunk in chunks:
            texts.append(chunk)
            metadatas.append({"source": doc_name})
    except Exception as e:
        logger.error(f"Error processing {doc_name}: {e}")

# Add Q&A data to the embeddings
for question, answer, source in zip(questions, answers, sources):
    texts.append(question)
    metadatas.append({"answer": answer, "source": source})

# Create embeddings and Chroma vectorstore
if texts:
    embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

    # Combine document text and Q&A into the vectorstore
    vectorstore = Chroma.from_texts(
        texts=texts,
        embedding=embeddings,
        metadatas=metadatas,
        persist_directory=DB_PATH
    )
    vectorstore.persist()
    logger.info("✅ Chroma vectorstore created and persisted!")
else:
    logger.warning("No text found to embed. Please ensure your documents are text-based (not scanned images).")


<ipython-input-7-ac3a553c779e>:46: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x788a71674600>
<ipython-input-7-ac3a553c779e>:55: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
def clean_redundancy(text):
    """
    Remove repeated phrases and sentences from the response.
    """

    sentences = text.split(". ")
    seen = set()
    cleaned = [sentence for sentence in sentences if sentence not in seen and not seen.add(sentence)]
    return ". ".join(cleaned).strip()

def answer_question(question, vectorstore, top_k=3, max_new_tokens=300, max_length=512):
    """
    Generate a concise, non-redundant, and complete answer using Falcon.
    """

    normalized_query = question.strip().lower()


    for i, original_question in enumerate(questions):
        if normalized_query == original_question.strip().lower():
            return f"{answers[i]} (Source: {sources[i]})"


    docs = vectorstore.similarity_search(query=normalized_query, k=top_k)
    if not docs:
        return "Sorry, I couldn't find relevant information for your question."


    unique_contexts = list({doc.page_content[:500] for doc in docs})
    if not unique_contexts:
        return "Sorry, no relevant information was found for your query."
    context_str = "\n\n".join(unique_contexts[:3])  # Use only the top 3 chunks for context

    # Build the prompt
    prompt = (
        "You are a helpful legal assistant.\n\n"
        f"Context:\n{context_str}\n\n"
        f"Question: {question}\n\n"
        "Please summarize the relevant details concisely, without repeating ideas. Provide a complete and clear answer directly addressing the question.\n\n"
        "Answer:"
    )


    inputs = tokenizer(prompt, return_tensors="pt", max_length=max_length, truncation=True).to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.2,
        top_p=0.9,
        do_sample=True
    )

    # Decode and clean the answer
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in generated_text:
        answer = generated_text.split("Answer:", 1)[1].strip()
    else:
        answer = generated_text.strip()

    return clean_redundancy(answer)


In [ ]:
# Ensure vectorstore is properly initialized
if vectorstore:

    response = answer_question("hasan case", vectorstore)
    print("Answer:", response)
else:
    print("Vectorstore is not initialized. Ensure documents have been processed correctly.")

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: Yes, the case of Al Hassan v. Egypt (Prosecutor v. Al Hassan v. Egypt) is a case before the International Criminal Court (ICC) involving the alleged commission of war crimes and crimes against humanity by the Egyptian military and security forces in Egypt's 2011 counter-insurgency operations in the Sinai Peninsula. The case was initiated by the Prosecutor of the International Criminal Court (ICC) on 17 February 2021, and the case is ongoing. The case involves the introduction of prior recorded testimonies pursuant to Rule 68(3) of the Rules of the International Criminal Court. The case is significant because it is the first case before the ICC to address the alleged commission of war crimes and crimes against humanity by the Egyptian military and security forces in the Sinai Peninsula.


In [ ]:
!zip -r project.zip /content


  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.01.15/ (stored 0%)
  adding: content/.config/logs/2025.01.15/14.25.10.736082.log (deflated 57%)
  adding: content/.config/logs/2025.01.15/14.25.02.096828.log (deflated 58%)
  adding: content/.config/logs/2025.01.15/14.25.00.998106.log (deflated 87%)
  adding: content/.config/logs/2025.01.15/14.25.11.351475.log (deflated 56%)
  adding: content/.config/logs/2025.01.15/14.24.33.570214.log (deflated 93%)
  adding: content/.config/logs/2025.01.15/14.24.53.360583.log (deflated 58%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/h

In [ ]:
from google.colab import files
files.download("project.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


In [19]:
import gradio as gr

def gradio_interface(question):
    """
    Gradio wrapper for the answer_question function.
    """
    if not question.strip():
        return "Please enter a question to proceed."

    try:
        response = answer_question(question, vectorstore)
        return response
    except Exception as e:
        return f"An error occurred: {str(e)}"

# Footer content
footer_content = """
<div style="text-align: center; margin-top: 20px; color: red;">
    <h4>Disclaimer</h4>
    <p>
        This application uses Falcon-7B Instruct, a large language model, to generate responses. While LLMs are highly capable,
        they may produce inaccurate, irrelevant, or outdated information. Use outputs responsibly and verify any critical details independently.
    </p>
    <p>
        This chatbot was trained and developed as a part of the <strong>AI + Digital Evidence Hackathon</strong>.
    </p>
    <p>
        <strong>Model:</strong> Falcon-7B Instruct | <strong>Embeddings:</strong> Sentence-Transformers (all-MiniLM-L6-v2)
        <strong>Libraries:</strong> Transformers, LangChain, Chroma, Gradio
        <strong>Key Parameters:</strong> max_new_tokens=300, temperature=0.2, top_p=0.9
    </p>
</div>
"""

# Use Gradio Blocks for layout customization
with gr.Blocks(css=".gradio-container { font-family: Arial, sans-serif; }") as app:
    gr.Markdown(
        """
        <div style="text-align: center; color: blue;">
            <h1>Lawbotica: Your Legal Document Assistant</h1>
            <p>
                <br>This assistant was trained on legal documents, including those available at
                <a href="https://leiden-guidelines.com" target="_blank">Leiden Guidelines</a>.
            </p>
        </div>
        """
    )
    with gr.Row():
        with gr.Column(scale=1, min_width=400):
            question_input = gr.Textbox(
                label="Enter Your Legal Question",
                placeholder="Ask a question about your documents...",
                lines=10  # Increased size
            )
        with gr.Column(scale=1, min_width=400):
            answer_output = gr.Textbox(
                label="Answer",
                placeholder="The assistant's answer will appear here.",
                lines=10  # Increased size
            )
    gr.Button("Submit", elem_id="submit-btn").click(gradio_interface, inputs=question_input, outputs=answer_output)
    gr.Markdown(footer_content)

# Add custom CSS for blue styling
app.css = """
#submit-btn {
    background-color: blue;
    color: white;
    font-weight: bold;
    border-radius: 5px;
    padding: 10px 15px;
}
"""

# Launch Gradio app
app.launch(share=True)








/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=177, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()
/usr/local/lib/python3.11/dist-packages/gradio/http_server.py:120: ResourceWarning: unclosed <socket.socket fd=178, family=2, type=1, proto=0, laddr=('0.0.0.0', 0)>
  s = socket.socket()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9937a61680700ad438.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
